In [54]:
import pandas as pd
import MultiContagion as mc
import igraph
import random
import numpy as np
from matplotlib import pylab as plt
import scipy.stats
import copy
from scipy.sparse import linalg as LA
#import powerlaw
%matplotlib inline

In [55]:
year = 2015

In [56]:
#import the aggregate adjacency matrix
aggregate_am = np.genfromtxt ('../data/adj/AM4_all_nodes_aggregate'+str(year)+'.csv', delimiter=",")
df_names = pd.read_csv('../data/adj/all_country_name4.csv', header=None)
names = list(df_names[0])
Aggregate_g = igraph.Graph.Weighted_Adjacency(list(np.transpose(aggregate_am)))     ## 
Aggregate_g.vs["name"] = copy.deepcopy(names)

In [57]:
#Import the adjacency matrix and make graphs of all the layers.
cdis_equity_am = np.genfromtxt ('../data/adj/AM4_all_nodesCDIS-equity'+str(year)+'.csv', delimiter=",")
cdis_equity_g = igraph.Graph.Weighted_Adjacency(list(cdis_equity_am))
cdis_equity_g.vs["name"] = copy.deepcopy(names)

cdis_debt_am = np.genfromtxt ('../data/adj/AM4_all_nodesCDIS-debt'+str(year)+'.csv', delimiter=",")
cdis_debt_g = igraph.Graph.Weighted_Adjacency(list(cdis_debt_am))
cdis_debt_g.vs["name"] = copy.deepcopy(names)

cpis_equity_am = np.genfromtxt ('../data/adj/AM4_all_nodesCPIS-equity'+str(year)+'.csv', delimiter=",")
cpis_equity_g = igraph.Graph.Weighted_Adjacency(list(cpis_equity_am))
cpis_equity_g.vs["name"] = copy.deepcopy(names)

cpis_debt_am = np.genfromtxt ('../data/adj/AM4_all_nodesCPIS-debt'+str(year)+'.csv', delimiter=",")
cpis_debt_g = igraph.Graph.Weighted_Adjacency(list(cpis_debt_am))
cpis_debt_g.vs["name"] = copy.deepcopy(names)

bis_am = np.genfromtxt ('../data/adj/AM4_all_nodesBIS'+str(year)+'.csv', delimiter=",")
bis_g = igraph.Graph.Weighted_Adjacency(list(bis_am))
bis_g.vs["name"] = copy.deepcopy(names)

#### we created two sets of adjmatrix, one set 1 is for cash flow, set 2 is for risk flow. the default adj matrix is for risk flow

In [58]:
G_list = [cdis_equity_g, cdis_debt_g, cpis_equity_g, cpis_debt_g, bis_g ]
AM_list = [np.transpose(cdis_equity_am), np.transpose(cdis_debt_am), 
           np.transpose(cpis_equity_am), np.transpose(cpis_debt_am), np.transpose(bis_am)]
G_list2 = [cdis_equity_g,cpis_equity_g, cpis_debt_g, bis_g ]
AM_list2 = [cdis_equity_am, cpis_equity_am, cpis_debt_am, bis_am ]

In [59]:
In_list = [ np.array(g.strength(mode ="IN", weights= g.es["weight"])) for g in G_list]
Out_list = [ np.array(g.strength(mode ="OUT", weights= g.es["weight"])) for g in G_list]

In [60]:
def make_supra_coup(list_adj, list_vector):
    "Takes a list of adjacency matrix and list of vectors to use as coupling for layers"
    n_layers = len(list_adj)
    dim = list_adj[0].shape[0]
    sup_list = []
    for i in range(n_layers):
        row = []
        for j in range(n_layers):
            if i == j:
                row.append(list_adj[i])
            else:
                row.append(np.diag(list_vector[j]))
        sup_list.append(row)
    return np.bmat(sup_list)

# For real

In [77]:
supra_adj = make_supra_coup(AM_list, In_list)
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())
complete_multi_pr = G_supra.personalized_pagerank(weights = G_supra.es["weight"])

In [84]:
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())

In [85]:
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())

In [86]:
G_supra.is_weighted(), G_supra.is_directed()

(True, True)

In [87]:
def save_centrality(y, namefile, namecent , country, cent):
    f = open("../data/centralities/"+namefile+str(y) +".csv", "w")
    f.write("Country,"+namecent+"\n")
    n = len(country)
    for i in range(n):
        f.write(str(country[i]) + "," + str(cent[i])+ "\n")
    f.close()

## PageRank

In [88]:
pr_agg = Aggregate_g.personalized_pagerank(weights = Aggregate_g.es["weight"], damping = 0.85)
node_names = np.array(copy.deepcopy(names))
inds = np.array(pr_agg).argsort()[::-1][:]
sort_names = node_names[inds]
pr_agg_ord = np.array(pr_agg)[inds]

In [89]:
sort_names[:30]

array(['United States', 'United Kingdom', 'Luxembourg', 'Netherlands',
       'France', 'Germany', 'Cayman Islands', 'China  P.R.: Hong Kong',
       'Switzerland', 'China  P.R.: Mainland', 'Japan', 'Ireland',
       'Canada', 'Italy', 'Spain', 'Australia', 'Belgium', 'Singapore',
       'India', 'Bermuda', 'Brazil', 'Sweden', 'Jersey',
       'Virgin Islands  British', 'Mexico', 'Austria',
       'Korea  Republic of', 'Mauritius', 'Russian Federation', 'Denmark'], 
      dtype='<U36')

In [90]:
save_centrality(year, "Agg_PageRank", "PageRank", sort_names, pr_agg_ord)

In [91]:
def get_pagerank_list(G, n_layers = 5, names= names, r = 0.85):
    pr = G.personalized_pagerank(weights = G.es["weight"], damping = r)
    multi_pr = []
    n_nodes = int((len(G.degree()))/5)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            cent += pr[i + n_nodes*k] 
        multi_pr.append(cent)
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [21]:
n_pr, num_pr = get_pagerank_list(G_supra, r = 0.85)

In [22]:
n_pr, num_pr = get_pagerank_list(G_supra, r = 0.85)
n_pr[:30]

array(['United States', 'United Kingdom', 'France', 'Luxembourg',
       'Switzerland', 'China  P.R.: Hong Kong', 'Netherlands',
       'China  P.R.: Mainland', 'Germany', 'Ireland', 'Japan',
       'Cayman Islands', 'Australia', 'Spain', 'Belgium', 'Italy',
       'Canada', 'India', 'Jersey', 'Singapore', 'Austria',
       'Korea  Republic of', 'Sweden', 'Portugal', 'Mauritius', 'Denmark',
       'Bermuda', 'Panama', 'Mexico', 'Turkey'], 
      dtype='<U36')

In [23]:
def make_supra_coup2(list_adj, list_vector):
    "Takes a list of adjacency matrix and list of vectors to use as coupling for layers"
    n_layers = len(list_adj)
    dim = list_adj[0].shape[0]
    sup_list = []
    for i in range(n_layers):
        row = []
        for j in range(n_layers):
            if i == j:
                row.append(list_adj[i])
            else:
                row.append(np.diag(list_vector[i]))
        sup_list.append(row)
    return np.bmat(sup_list)

In [24]:
supra_adj2 = make_supra_coup2(AM_list, In_list)
G_supra2 = igraph.Graph.Weighted_Adjacency(supra_adj2.tolist())

In [25]:
n_pr2, num_pr2 = get_pagerank_list(G_supra2)

In [26]:
n_pr2;

In [27]:
n_pr2, num_pr2 = get_pagerank_list(G_supra2, r = 0.85)
n_pr2[:30]

array(['United States', 'United Kingdom', 'Netherlands', 'Luxembourg',
       'France', 'Germany', 'China  P.R.: Mainland',
       'China  P.R.: Hong Kong', 'Cayman Islands', 'Switzerland', 'Japan',
       'Ireland', 'Italy', 'Canada', 'Mauritius', 'Australia', 'Spain',
       'Belgium', 'India', 'Brazil', 'Russian Federation', 'Singapore',
       'Mexico', 'Austria', 'Sweden', 'Korea  Republic of', 'Thailand',
       'Panama', 'South Africa', 'Bermuda'], 
      dtype='<U36')

In [28]:
save_centrality(year, "MultCoupCashFlow_PageRank", "PageRank centrality", n_pr, num_pr)

In [29]:
save_centrality(year, "MultCoupRiskFlow_PageRank", "PageRank centrality", n_pr2, num_pr2)

## Eigenvector

In [30]:
def get_eigenvector_list(G, n_layers = 5, names= names):
    pr = G.eigenvector_centrality(weights = G.es["weight"])
    multi_pr = []
    n_nodes = int((len(G.degree()))/5)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            cent += pr[i + n_nodes*k] 
        multi_pr.append(cent)
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [31]:
n_ei, num_ei = get_eigenvector_list(G_supra)
n_ei2, num_ei2 = get_eigenvector_list(G_supra2)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: Weighted directed graph in eigenvector centrality at centrality.c:362
  from ipykernel import kernelapp as app


In [32]:
n_ei2;

In [33]:
save_centrality(year, "MultCoupCashFlow_Eigenvector", "Eigenvector centrality", n_ei, num_ei)
save_centrality(year, "MultCoupRiskFlow_Eigenvector", "Eigenvector centrality", n_ei2, num_ei2)

## Hubs and Auth

In [34]:
def G_list_hub(G_adj_list):
    G_hub = []
    for g in G_adj_list:
        new = np.dot(g, g.transpose())
        G_hub.append(new)   
    return G_hub

In [35]:
def G_list_auth(G_adj_list):
    G_hub = []
    for g in G_adj_list:
        new = np.dot( g.transpose(), g)
        G_hub.append(new)   
    return G_hub

In [36]:
def hub_auth_diag(Coup_list):
    coup = []
    for c in Coup_list:
        coup.append(c*c)
    return coup

In [37]:
G_supra_hub = make_supra_coup(G_list_hub(AM_list), hub_auth_diag(In_list))
G_supra_auth = make_supra_coup(G_list_auth(AM_list), hub_auth_diag(In_list))

In [38]:
G_supra_hub[0].shape

(1, 1065)

In [39]:
def get_hub_list(AM, n_layers = 5, names= names):
    hub_eig = LA.eigs(AM, k=1, which='LM')[1]
    multi_pr = []
    #print hub_eig[0]
    n_nodes = int((AM[0].shape[1])/5.)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            
            cent += abs(hub_eig[i + n_nodes*k])
        
        multi_pr.append(cent[0])
    
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [40]:
n_hub, num_hub = get_hub_list(G_supra_hub)
n_auth, num_auth = get_hub_list(G_supra_auth)

In [41]:
n_auth, num_auth = get_hub_list(G_supra_auth)

In [42]:
n_auth[:20]

array(['United States', 'United Kingdom', 'Netherlands', 'France',
       'Luxembourg', 'Germany', 'Ireland', 'Cayman Islands', 'Switzerland',
       'Japan', 'Italy', 'Belgium', 'Spain', 'Canada', 'Australia',
       'China  P.R.: Hong Kong', 'China  P.R.: Mainland', 'Singapore',
       'Sweden', 'Brazil'], 
      dtype='<U36')

In [43]:
n_hub[:20]

array(['United States', 'Luxembourg', 'United Kingdom', 'Netherlands',
       'Germany', 'Japan', 'France', 'China  P.R.: Hong Kong',
       'Switzerland', 'Ireland', 'Belgium', 'Bermuda', 'Canada', 'Italy',
       'Cayman Islands', 'Jersey', 'Norway', 'Spain', 'Singapore', 'Sweden'], 
      dtype='<U36')

In [44]:
save_centrality(year, "MultCoupCashFlow_Hub", "Hub Score", n_hub, num_hub)
save_centrality(year, "MultCoupCashFlow_Auth", "Authority Score", n_auth, num_auth)

In [45]:
n_auth[:20]

array(['United States', 'United Kingdom', 'Netherlands', 'France',
       'Luxembourg', 'Germany', 'Ireland', 'Cayman Islands', 'Switzerland',
       'Japan', 'Italy', 'Belgium', 'Spain', 'Canada', 'Australia',
       'China  P.R.: Hong Kong', 'China  P.R.: Mainland', 'Singapore',
       'Sweden', 'Brazil'], 
      dtype='<U36')

In [46]:
n_hub[:20]

array(['United States', 'Luxembourg', 'United Kingdom', 'Netherlands',
       'Germany', 'Japan', 'France', 'China  P.R.: Hong Kong',
       'Switzerland', 'Ireland', 'Belgium', 'Bermuda', 'Canada', 'Italy',
       'Cayman Islands', 'Jersey', 'Norway', 'Spain', 'Singapore', 'Sweden'], 
      dtype='<U36')

### other coup

In [47]:
G_supra_hub2 = make_supra_coup2(G_list_hub(AM_list), hub_auth_diag(In_list))
G_supra_auth2 = make_supra_coup2(G_list_auth(AM_list), hub_auth_diag(In_list))

In [48]:
n_hub2, num_hub2 = get_hub_list(G_supra_hub2)
n_auth2, num_auth2 = get_hub_list(G_supra_auth2)

In [49]:
#n_hub2, num_hub2 = get_eigenvector_list(igraph.Graph.Weighted_Adjacency(G_supra_hub2.tolist()))
#n_auth2, num_auth2 = get_eigenvector_list(igraph.Graph.Weighted_Adjacency(G_supra_auth2.tolist()))

In [50]:
n_auth2[:30]

array(['United States', 'United Kingdom', 'Netherlands', 'Luxembourg',
       'France', 'Ireland', 'Germany', 'Cayman Islands', 'Switzerland',
       'Japan', 'Belgium', 'Italy', 'Spain', 'Canada', 'Australia',
       'China  P.R.: Mainland', 'China  P.R.: Hong Kong', 'Bermuda',
       'Brazil', 'Singapore', 'Sweden', 'Korea  Republic of', 'Cyprus',
       'Jersey', 'India', 'Mexico', 'South Africa', 'Denmark', 'Gibraltar',
       'Taiwan Province of China'], 
      dtype='<U36')

In [51]:
n_hub2[:30]

array(['United States', 'Luxembourg', 'United Kingdom', 'Japan',
       'Netherlands', 'Germany', 'China  P.R.: Hong Kong', 'France',
       'Switzerland', 'Canada', 'Ireland', 'Italy', 'Bermuda', 'Belgium',
       'Norway', 'Spain', 'Sweden', 'Singapore', 'Jersey',
       'China  P.R.: Mainland', 'South Africa', 'Cayman Islands',
       'Australia', 'Denmark', 'Guernsey', 'Finland',
       'Virgin Islands  British', 'Gibraltar', 'Korea  Republic of',
       'Austria'], 
      dtype='<U36')

In [52]:
save_centrality(year, "MultCoupRiskFlow_Hub", "Hub Score", n_hub2, num_hub2)
save_centrality(year, "MultCoupRiskFlow_Auth", "Authority Score", n_auth2, num_auth2)

In [53]:
num_hub2 [:20]

array([  1.75162684e+00,   2.28981910e-02,   1.40877055e-02,
         1.18188042e-02,   1.17787839e-02,   9.21828951e-03,
         7.49188193e-03,   6.92733554e-03,   4.91752008e-03,
         3.74162955e-03,   3.31115658e-03,   3.13144791e-03,
         3.07965051e-03,   2.80933093e-03,   2.78902181e-03,
         1.95752617e-03,   1.89013740e-03,   1.73176617e-03,
         1.61466686e-03,   1.19585050e-03])